In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from models.dnn import DNN
from preprocessing.encoding import seqToTensor, sequencesToTensor

%load_ext autoreload
%autoreload 2

In [3]:
from torch.utils.data import DataLoader
from preprocessing.csv_dataset import CSVDataset

# Load Data

In [4]:
# Just a quick glimpse at the data

In [5]:
data = pd.read_csv("data/preprocessed-1-dnn.csv")
data

,TrainSeq,LastNum
0,"1,1,9,1,25,11,1,49,31,43,1,81,61,247,85,1,121,...",000657871
1,"1,0,0,1,0,0,1,0,0,1,0,0,2,0,0,2,0,0,2,0,0,2,0,...",000000000
2,"1,1,1,2,1,1,2,1,1,2,2,2,2,2,3,2,3,3,2,2,3,1,1,...",000000003
3,"0,1,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,...",000000009
4,"3,8,4,5,7,1,7,5,0,5,0,4,9,4,8,7,0,3,1,4,7,9,4,...",000000007
...,...,...
3871,"1,1,1,1,2,1,1,3,3,1,1,4,6,4,1,1,6,10,10,6,1,1,...",000021259
3872,"2,7,6,8,6,7,8,2,8,2,9,8,7,3,2,1,5,3,2,4,9,5,3,...",000000006
3873,"1,1,1,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,...",000000005
3874,"1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",000000002


In [6]:
data['TrainSeq']

0       1,1,9,1,25,11,1,49,31,43,1,81,61,247,85,1,121,...
1       1,0,0,1,0,0,1,0,0,1,0,0,2,0,0,2,0,0,2,0,0,2,0,...
2       1,1,1,2,1,1,2,1,1,2,2,2,2,2,3,2,3,3,2,2,3,1,1,...
3       0,1,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,...
4       3,8,4,5,7,1,7,5,0,5,0,4,9,4,8,7,0,3,1,4,7,9,4,...
                              ...                        
3871    1,1,1,1,2,1,1,3,3,1,1,4,6,4,1,1,6,10,10,6,1,1,...
3872    2,7,6,8,6,7,8,2,8,2,9,8,7,3,2,1,5,3,2,4,9,5,3,...
3873    1,1,1,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,...
3874    1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...
3875    1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,...
Name: TrainSeq, Length: 3876, dtype: object

In [7]:
input_length = len(data['TrainSeq'][0])
encoded = []
symbols = '0123456789,-'

for sequence in data['TrainSeq']:
    flattened = torch.flatten(seqToTensor(sequence, input_length, symbols = symbols))
    encoded.append(flattened)

KeyboardInterrupt: 

In [15]:
# Load it in an efficient way

In [16]:
train_dataset = CSVDataset('data/preprocessed-1-dnn.csv')
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Define Model

In [17]:
''' Define key attributes '''
#input_size = encoded[0].shape[0]
input_size = len(data['TrainSeq'][0])#*len(symbols)
output_size = 9 #* len(symbols)

In [18]:
model = DNN(input_size, output_size, symbol_set = symbols,seed = 5, hidden_layer_param=[640])
print(model)

DNN(
  (fc_in): Linear(in_features=207, out_features=640, bias=True)
  (fc_out): Linear(in_features=640, out_features=9, bias=True)
)


# Training

In [25]:
def mycriterion(output, target):
    max_indexes = torch.argmax(output, dim=1)

    print(max_indexes)

    loss = torch.sub(output, target)
    return loss


In [26]:
def run_training(model, train_dataloader, epochs=1):
    criterion = mycriterion #nn.MSELoss() #lambda x, y: (x-y)**2
    # criterion = nn.CrossEntropyLoss() #lambda x, y: (x-y)**2
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            encoded_inputs = sequencesToTensor(inputs, symbols = symbols)
            encoded_inputs = encoded_inputs.transpose(1,2)
            # print(encoded_inputs.shape)

            encoded_labels = sequencesToTensor(labels, symbols = symbols)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(encoded_inputs)

            # print(outputs.shape)
            # print(encoded_labels.shape)
            
            # print(outputs)
            transposed_outputs = outputs.transpose(1,2)
            # transposed_labels = encoded_labels.transpose(1,2)

            # print(transposed_outputs.shape)
            # print(transposed_labels.shape)
            loss = criterion(transposed_outputs, encoded_labels)
            
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

            # if i % 2000 == 1999:    # print every 2000 mini-batches
            # print('[%d, %5d] loss: %.3f' %
            #     (epoch + 1, i + 1, loss.item()))
            # running_loss = 0.0

    print('Finished Training')

In [27]:
run_training(model, train_dataloader, epochs=1)

tensor([[0, 8, 0, 0, 1, 0, 0, 0, 0, 0, 5, 0]])


RuntimeError: grad can be implicitly created only for scalar outputs

# Testing it

In [47]:
def test_run(model, test_dataloader):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in test_dataloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

    return 100 * correct / total

In [48]:
test_run(model, train_dataloader)

AttributeError: 'tuple' object has no attribute 'dim'